In [1]:
import tensorflow as tf
from tensorflow.keras.models import load_model
import pickle
import pandas as pd
import numpy as np

In [6]:
## Load the train model, scaler, one hot
model=load_model('model.h5')

#load the encoders and scaler

with open('label_encoder_gender.pkl','rb') as file:
    label_encoder_gender = pickle.load(file)

with open('one_hot_geo.pkl','rb') as file:
    one_hot_geo = pickle.load(file)

with open('scaler.pkl','rb') as file:
    scaler = pickle.load(file)

In [53]:
input_data = {
    'CreditScore':600,
    'Geography':'France',
    'Gender':'Male',
    'Age':40,
    'Tenure':3,
    'Balance':60000,
    'NumOfProducts':2,
    'HasCrCard':1,
    'IsActiveMember':1,
    'EstimatedSalary':50000
}

In [54]:
geo_encoded = one_hot_geo.transform([[input_data['Geography']]]).toarray()
geo_encoded_df = pd.DataFrame(geo_encoded,columns=one_hot_geo.get_feature_names_out(['Geography']))
geo_encoded_df

c:\Users\Manoj Kumar\Desktop\DS\ANN classifier project\venv\Lib\site-packages\sklearn\utils\validation.py:2749: UserWarning: X does not have valid feature names, but OneHotEncoder was fitted with feature names
  warnings.warn(


,Geography_France,Geography_Germany,Geography_Spain
0,1.0,0.0,0.0


In [55]:
input_df = pd.DataFrame([input_data])
input_df

,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary
0,600,France,Male,40,3,60000,2,1,1,50000


In [56]:
input_df['Gender'] = label_encoder_gender.transform(input_df['Gender'])
input_df

,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary
0,600,France,1,40,3,60000,2,1,1,50000


In [57]:
## Concat OnehotEncoder and 
input_df = pd.concat([input_df.drop('Geography',axis=1),geo_encoded_df],axis=1)
input_df

,CreditScore,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Geography_France,Geography_Germany,Geography_Spain
0,600,1,40,3,60000,2,1,1,50000,1.0,0.0,0.0


In [58]:
input_scaled =  scaler.transform(input_df)
input_scaled

array([[-0.54007722,  0.91186722,  0.10685843, -0.69164931, -0.26034447,
         0.81311138,  0.64946204,  0.96978946, -0.88017893,  1.00053348,
        -0.57776083, -0.57735027]])

In [61]:
## Predict Churn
prediction = model.predict(input_scaled)
prediction_prob = prediction[0][0]

1/1 [==============================] - 0s 20ms/step


In [62]:
prediction_prob

0.024634276

In [63]:
if prediction_prob>0.5:
    print('The customer is likely to churn')
else:
    print('The customer is not likely to churn')


The customer is not likely to churn
